#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/mattgordon/Documents/Programming/Learning/Data Engineering Nanodegree/Projects/Project 2


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print("Number of rows in csv file: " + str(sum(1 for line in f)))

Number of rows in csv file: 6821


The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Create a Cluster

In [5]:
# This should make a connection to a Cassandra instance 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute(""" CREATE KEYSPACE IF NOT EXISTS sparkify WITH REPLICATION = 
                    {'class':'SimpleStrategy','replication_factor':1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exceoption as e:
    print(e)

## Requested Queries
The following queries have been requested by the Sparkify Analytics Team.

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338 and itemInSession = 4 

In order to generate data for this query, we need to create the following table using sessionId as a Partition Key and itemInSession as a Clustering Key.
    
| Column | Variable      | Type    |
|--------|---------------|---------|
| 1      | sessionId     | int     |
| 2      | itemInSession | int     |
| 3      | artist_name   | text    |
| 4      | song_title    | text    |
| 5      | song_length   | decimal |   
   

In [8]:
# Create songs_played_in_session table
query = "CREATE TABLE IF NOT EXISTS songs_played_in_session"
query = query + """(sessionId int, itemInSession int, artist_name text, song_title text, song_length decimal,
                    PRIMARY KEY(sessionId, itemInSession))"""

try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
# Populate the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Insert data into Cassandra table
        query = """INSERT INTO songs_played_in_session (sessionId, itemInSession, artist_name, song_title, 
                    song_length)"""
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]),line[0],line[9],float(line[5])))

In [10]:
#  Verify the data was entered into the table
query =  """select artist_name, song_title, song_length FROM songs_played_in_session WHERE sessionId = 338 
            AND itemInSession = 4"""

# Create dataframe for results
df = pd.DataFrame(columns=['artist name','song title','song length'])

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    df = df.append({'artist name':row.artist_name,'song title':row.song_title,'song length':row.song_length}, ignore_index=True)
df

,artist name,song title,song length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In order to generate data for this query, we need to create the following table using userId as a Partition Key and sessionId as a Clustering Key.   

| Column | Variable      | Type |
|--------|---------------|------|
| 1      | userId        | int  |
| 2      | sessionId     | int  |
| 3      | itemInSession | int  |
| 4      | song_title    | text |
| 5      | artist_name   | text |
| 6      | user_name     | text |    
    

In [11]:
# Create songs_played_by_user table
query = """CREATE TABLE IF NOT EXISTS songs_played_by_user"""
query = query + """(userId int, sessionId int,itemInSession int, song_title text, artist_name text,  
                user_name text ,PRIMARY KEY(userId, sessionId)); 
                """
try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
# Populate the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Insert data into Cassandra table
        query = """INSERT INTO songs_played_by_user (userId, sessionId, itemInSession, song_title, artist_name, 
                user_name)"""
        query = query + "VALUES (%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]), int(line[8]),int(line[3]),line[9],line[0],line[1] + ' ' + line[4]))

In [13]:
#  Verify the data was entered into the table
query =  "select artist_name, song_title, user_name FROM songs_played_by_user WHERE userId = 10 AND sessionId = 182"

# Create dataframe for results
df = pd.DataFrame(columns=['artist name','song title','user name'])

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    df = df.append({'artist name':row.artist_name,'song title': row.song_title, 'user name': row.user_name}, ignore_index=True) 
df

,artist name,song title,user name
0,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In order to generate data for this query, we need to create the following table using song_title as a Partition Key and user_name as a Clustering Key.   
    
| Column | Variable   | Type |
|--------|------------|------|
| 1      | song_title | text |
| 2      | user_name  | text |   


In [14]:
# Create listeners_of_song table
query = "CREATE TABLE IF NOT EXISTS listeners_of_song"
query = query + "(song_title text, user_name text, PRIMARY KEY(song_title, user_name));"

try:
    session.execute(query)
except Exception as e:
    print(e)   

In [15]:
# Populate the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Insert data into Cassandra table
        query = "INSERT INTO listeners_of_song (song_title, user_name)"
        query = query + "VALUES (%s,%s)"
        session.execute(query, (line[9], line[1] + ' ' + line[4]))

In [16]:
#  Verify the data was entered into the table
query =  "select user_name FROM listeners_of_song WHERE song_title = 'All Hands Against His Own'"

# Create dataframe for results
df = pd.DataFrame(columns=['user name'])

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    df = df.append({'user name':row.user_name}, ignore_index=True)  
df

,user name
0,Jacqueline Lynch
1,Sara Johnson
2,Tegan Levine


### Drop the tables before closing out the sessions

In [17]:
# Drop table statements
drop_songs_played_in_session = "drop table songs_played_in_session"
drop_songs_played_by_user = "drop table songs_played_by_user"
drop_listeners_of_song = "drop table listeners_of_song"

drop_all_tables = [drop_songs_played_in_session, drop_songs_played_by_user, drop_listeners_of_song]

# Loop through drop statements and execute
for table in drop_all_tables:
    try:
        session.execute(table)
    except Exception as e:
        print(e)

### Close the session and cluster connection

In [18]:
session.shutdown()
cluster.shutdown()